I haven't done this in a while. It used to be Yahoo Finance API, then Quantopian and Quandl, I'm way behind. I hear good things about _Alpha Vantage_.

In [9]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from functools import lru_cache

load_dotenv()

@lru_cache(maxsize=None)
def fetch_dailies(ticker):
  url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={os.getenv('ALPHAVANTAGE_API_KEY')}"
  response = requests.get(url)
  response = response.json()
  # Convert the response to a pandas DataFrame
  df = pd.DataFrame(response['Time Series (Daily)']).T
  df.index = pd.to_datetime(df.index)
  df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
  df['Ticker'] = ticker  # Add a new column 'Ticker' with the ticker value
  return df


In [10]:
fetch_dailies('AAPL')

,Open,High,Low,Close,Volume,Ticker
2024-02-28,182.5100,183.1200,180.1300,181.4200,48953939,AAPL
2024-02-27,181.1000,183.9225,179.5600,182.6300,54318851,AAPL
2024-02-26,182.2400,182.7600,180.6500,181.1600,40867421,AAPL
2024-02-23,185.0100,185.0400,182.2300,182.5200,45119677,AAPL
2024-02-22,183.4800,184.9550,182.4600,184.3700,52292208,AAPL
...,...,...,...,...,...,...
2023-10-11,178.2000,179.8500,177.6000,179.8000,47551098,AAPL
2023-10-10,178.1000,179.7200,177.9500,178.3900,43698019,AAPL
2023-10-09,176.8100,179.0500,175.8000,178.9900,42390772,AAPL
2023-10-06,173.8000,177.9900,173.1800,177.4900,57266675,AAPL


Easy, awesome, I'm game.